In [1]:
import datetime
import pandas as pd

In [2]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

In [3]:
%run "/workspaces/kiteconnect/seleting_instruments.ipynb"

%run "/workspaces/kiteconnect/Login_and_Mail_Alerts.ipynb"

%run "/workspaces/kiteconnect/Basket_and_direction_widgets.ipynb"

## LOGGING IN

In [4]:
kite = kite_login(api_key="kvvzk6jq4zn4o4vc",    api_secret="nzcqxwu05jmo2k71cmpgnbf5trz6gypn")

HTML(value='')

In [5]:
profile = kite.profile()

# Step 3: Print profile information
print("User ID:", profile["user_id"])
print("User Name:", profile["user_name"])
print("Email:", profile["email"])
print("User type:", profile["user_type"])
print("Products:", profile["products"])
print("Broker info:", profile["broker"])

User ID: FVC741
User Name: Balam Baji Babu
Email: balambajibabu143@gmail.com
User type: individual/res_no_nn
Products: ['CNC', 'NRML', 'MIS', 'BO', 'CO']
Broker info: ZERODHA


## CHART MONITOR

In [6]:
instruments = kite.instruments(exchange="NFO")

In [7]:
display(toggle)

ToggleButton(value=False, button_style='danger', description='Monitor OFF', icon='stop', tooltip='Toggle candl…

## PICKING DIRECTION

In [8]:
from datetime import datetime

weekday = datetime.today().weekday()

lp, up = (50, 70) if weekday in (0, 1) else (40, 60)
fut_symbols, option_symbols, tokens = pick_direction(kite, instruments, lp=lp, up=up)


# MAKING ORDER BASKET

In [9]:
make_order_basket(fut_symbols, option_symbols, kite)

Output()

Output()

## Placing Stop Loss Order for Spread Leg

In [17]:
exit_position_manager(kite)

Output()

Output()

Output()

Output()

## Modifying Orders

In [11]:
import pandas as pd

orders_df = pd.DataFrame(kite.orders())
orders_df

filtered_df = orders_df[~orders_df['status'].str.contains("CANCELLED", case=False, na=False)]


In [12]:
filtered_df[['order_id','status','tradingsymbol','order_type','transaction_type','validity','product','quantity','price','filled_quantity','pending_quantity']]

,order_id,status,tradingsymbol,order_type,transaction_type,validity,product,quantity,price,filled_quantity,pending_quantity
45,251207180050377,AMO REQ RECEIVED,NIFTY25DECFUT,MARKET,BUY,DAY,MIS,225,0,0,225
46,251207180051366,AMO REQ RECEIVED,NIFTY25DECFUT,MARKET,SELL,DAY,MIS,225,0,0,225


In [13]:
orders_df.columns

Index(['account_id', 'placed_by', 'order_id', 'exchange_order_id',
       'parent_order_id', 'status', 'status_message', 'status_message_raw',
       'order_timestamp', 'exchange_update_timestamp', 'exchange_timestamp',
       'variety', 'modified', 'exchange', 'tradingsymbol', 'instrument_token',
       'order_type', 'transaction_type', 'validity', 'validity_ttl', 'product',
       'quantity', 'disclosed_quantity', 'price', 'trigger_price',
       'average_price', 'filled_quantity', 'pending_quantity',
       'cancelled_quantity', 'market_protection', 'meta', 'tag', 'guid'],
      dtype='object')

In [14]:
fut_orders = orders_df[
    (~orders_df["status"].str.startswith("CANCELLED")) &
    (orders_df["tradingsymbol"].str.endswith("FUT"))
]


In [15]:
for _, row in fut_orders.iterrows():
    oid = row["order_id"]
    revised_price = 26340
    # revised_price = row['average_price']- initial_stoploss

    try:
        new_id = kite.modify_order(
            variety = "amo",
            order_id = oid,
            quantity = row["quantity"],
            order_type = "LIMIT",
            price = revised_price,
            trigger_price = revised_price,
            validity = "DAY"
        )

        print(f"MODIFIED: {oid} → new_id = {new_id}")

    except Exception as e:
        print(f"FAILED to modify {oid}: {e}")


MODIFIED: 251207180050377 → new_id = 251207180050377
MODIFIED: 251207180051366 → new_id = 251207180051366


## CANCELLING ORDERS

In [ ]:
orders_df = pd.DataFrame(kite.orders())


fut_orders = orders_df[
    (~orders_df["status"].str.startswith("CANCELLED")) &
    (orders_df["tradingsymbol"].str.endswith("FUT"))]


In [ ]:
for _, row in fut_orders.iterrows():
    oid = row["order_id"]

    try:
        kite.cancel_order(order_id=oid,
                          variety='amo')
        print(f"Order Cancellation Successful: {oid}")

    except Exception as e:
        print(f"FAILED to Cancel Order {oid}: {e}")


## EXIT ORDERS

In [ ]:
for _, row in fut_orders.iterrows():
    oid = row["order_id"]

    try:
        kite.exit_order(order_id=oid,
                          variety='amo')
        print(f"Order exited Successful: {oid}")

    except Exception as e:
        print(f"FAILED to Cancel Order {oid}: {e}")


In [ ]:
import pandas as pd

orders_df = pd.DataFrame(kite.orders())

# Keep only executed orders
executed_orders = orders_df[orders_df['status'].isin(['COMPLETE', 'TRIGGER PENDING'])]

executed_orders


In [ ]:
all_trades = []

for oid in fut_orders['order_id']:
    trades = kite.order_trades(order_id=oid)
    all_trades.extend(trades)

trades_df = pd.DataFrame(all_trades)
trades_df


# KITE TICKER

## This block printed once

In [ ]:
kws = KiteTicker(api_key=kite.api_key,   access_token=kite.access_token)

In [ ]:
tokens

In [ ]:

def on_connect(ws, response):
    print("Connected successfully")
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)

def on_ticks(ws, ticks):
    for tick in ticks:
        print(f"Time: {tick.get('timestamp')}")
        print(f"LTP: ₹{tick['last_price']}")
        print(f"Volume: {tick['volume']} lots")
        print(f"OI: {tick.get('oi', 0)} lots")
        print(f"Change: {tick.get('change', 0)}%")
        print("-" * 50)

def on_error(ws, code, reason):
    print(f"Error: {code} - {reason}")

def on_close(ws, code, reason):
    print("Connection closed")

# Assign callbacks
kws.on_connect = on_connect
kws.on_ticks = on_ticks
kws.on_error = on_error
kws.on_close = on_close

# Start ticker
kws.connect(threaded=True)

In [ ]:
kws.close()

In [ ]:
from kiteconnect import KiteTicker, KiteConnect
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class TrailingStopLoss:
    """
    Trailing Stop Loss System for Nifty December Future
    
    Features:
    - Automatic stop-loss adjustment as price moves up
    - Configurable trailing distance
    - Real-time position monitoring
    - Alert notifications when stop-loss is hit
    """
    
    def __init__(self, kite, kite_ticker, instrument_token):
        """
        Initialize trailing stop loss system
        
        Args:
            kite: KiteConnect instance for placing orders
            kite_ticker: KiteTicker instance for real-time data
            instrument_token: Instrument token for Nifty Dec Future
        """
        self.kite = kite
        self.kite_ticker = kite_ticker
        self.instrument_token = instrument_token
        
        # Position parameters
        self.entry_price = None
        self.current_price = None
        self.stop_loss = None
        self.highest_price = None
        self.position_active = False
        self.position_quantity = 0
        self.position_type = None  # 'BUY' or 'SELL'
        
        # Trailing parameters
        self.trailing_points = 15  # Default trailing distance
        self.initial_stop_distance = 15  # Initial stop loss distance
        
    def enter_position(self, entry_price, quantity, position_type='BUY', 
                      initial_stop_distance=15, trailing_points=15):
        """
        Enter a new position with initial stop loss
        
        Args:
            entry_price: Entry price for the position
            quantity: Number of lots/quantity
            position_type: 'BUY' for long, 'SELL' for short
            initial_stop_distance: Initial stop loss distance in points
            trailing_points: Trailing distance in points
        """
        self.entry_price = entry_price
        self.position_quantity = quantity
        self.position_type = position_type
        self.initial_stop_distance = initial_stop_distance
        self.trailing_points = trailing_points
        
        # Set initial stop loss
        if position_type == 'BUY':
            self.stop_loss = entry_price - initial_stop_distance
            self.highest_price = entry_price
        else:  # SELL position
            self.stop_loss = entry_price + initial_stop_distance
            self.highest_price = entry_price  # For short, track lowest price
        
        self.position_active = True
        
        logger.info(f"\n{'='*60}")
        logger.info(f"POSITION ENTERED")
        logger.info(f"{'='*60}")
        logger.info(f"Type: {position_type}")
        logger.info(f"Entry Price: ₹{entry_price}")
        logger.info(f"Quantity: {quantity}")
        logger.info(f"Initial Stop Loss: ₹{self.stop_loss}")
        logger.info(f"Trailing Distance: {trailing_points} points")
        logger.info(f"{'='*60}\n")
        
    def update_trailing_stop(self, current_price):
        """
        Update trailing stop loss based on current price
        
        Args:
            current_price: Current market price
        """
        if not self.position_active:
            return
        
        self.current_price = current_price
        old_stop_loss = self.stop_loss
        
        if self.position_type == 'BUY':
            # For long positions
            if current_price > self.highest_price:
                self.highest_price = current_price
                new_stop_loss = current_price - self.trailing_points
                
                # Only move stop loss up, never down
                if new_stop_loss > self.stop_loss:
                    self.stop_loss = new_stop_loss
                    logger.info(f"\n{'*'*60}")
                    logger.info(f"STOP LOSS UPDATED (LONG)")
                    logger.info(f"{'*'*60}")
                    logger.info(f"Current Price: ₹{current_price}")
                    logger.info(f"Highest Price: ₹{self.highest_price}")
                    logger.info(f"Old Stop Loss: ₹{old_stop_loss}")
                    logger.info(f"New Stop Loss: ₹{self.stop_loss}")
                    logger.info(f"Profit Protected: ₹{(self.stop_loss - self.entry_price) * self.position_quantity}")
                    logger.info(f"{'*'*60}\n")
            
            # Check if stop loss is hit
            if current_price <= self.stop_loss:
                self.exit_position("Stop Loss Hit")
                
        else:  # SELL position
            # For short positions, track lowest price
            if current_price < self.highest_price:  # highest_price is actually lowest for shorts
                self.highest_price = current_price
                new_stop_loss = current_price + self.trailing_points
                
                # Only move stop loss down, never up
                if new_stop_loss < self.stop_loss:
                    self.stop_loss = new_stop_loss
                    logger.info(f"\n{'*'*60}")
                    logger.info(f"STOP LOSS UPDATED (SHORT)")
                    logger.info(f"{'*'*60}")
                    logger.info(f"Current Price: ₹{current_price}")
                    logger.info(f"Lowest Price: ₹{self.highest_price}")
                    logger.info(f"Old Stop Loss: ₹{old_stop_loss}")
                    logger.info(f"New Stop Loss: ₹{self.stop_loss}")
                    logger.info(f"Profit Protected: ₹{(self.entry_price - self.stop_loss) * self.position_quantity}")
                    logger.info(f"{'*'*60}\n")
            
            # Check if stop loss is hit
            if current_price >= self.stop_loss:
                self.exit_position("Stop Loss Hit")
    
    def exit_position(self, reason="Manual Exit"):
        """
        Exit the position
        
        Args:
            reason: Reason for exit
        """
        if not self.position_active:
            logger.warning("No active position to exit")
            return
        
        exit_price = self.current_price
        
        if self.position_type == 'BUY':
            pnl = (exit_price - self.entry_price) * self.position_quantity
        else:
            pnl = (self.entry_price - exit_price) * self.position_quantity
        
        logger.info(f"\n{'='*60}")
        logger.info(f"POSITION EXITED")
        logger.info(f"{'='*60}")
        logger.info(f"Reason: {reason}")
        logger.info(f"Entry Price: ₹{self.entry_price}")
        logger.info(f"Exit Price: ₹{exit_price}")
        logger.info(f"Stop Loss: ₹{self.stop_loss}")
        logger.info(f"Highest Price Reached: ₹{self.highest_price}")
        logger.info(f"P&L: ₹{pnl:,.2f}")
        logger.info(f"P&L per lot: ₹{pnl/self.position_quantity:,.2f}")
        logger.info(f"{'='*60}\n")
        
        # Place exit order (uncomment when ready to use)
        # self.place_exit_order(exit_price)
        
        # Reset position
        self.position_active = False
        self.entry_price = None
        self.current_price = None
        self.stop_loss = None
        self.highest_price = None
        
    def place_exit_order(self, price):
        """
        Place exit order on Kite
        
        Args:
            price: Exit price
        """
        try:
            transaction_type = 'SELL' if self.position_type == 'BUY' else 'BUY'
            
            order_id = self.kite.place_order(
                variety=self.kite.VARIETY_REGULAR,
                exchange=self.kite.EXCHANGE_NFO,
                tradingsymbol="NIFTY24DECFUT",  # Update with actual symbol
                transaction_type=transaction_type,
                quantity=self.position_quantity,
                product=self.kite.PRODUCT_MIS,  # Or NRML for overnight
                order_type=self.kite.ORDER_TYPE_MARKET,
            )
            
            logger.info(f"Exit order placed. Order ID: {order_id}")
            
        except Exception as e:
            logger.error(f"Error placing exit order: {e}")
    
    def get_position_status(self):
        """Get current position status"""
        if not self.position_active:
            return "No active position"
        
        if self.position_type == 'BUY':
            unrealized_pnl = (self.current_price - self.entry_price) * self.position_quantity
        else:
            unrealized_pnl = (self.entry_price - self.current_price) * self.position_quantity
        
        status = f"""
        {'='*60}
        CURRENT POSITION STATUS
        {'='*60}
        Type: {self.position_type}
        Entry Price: ₹{self.entry_price}
        Current Price: ₹{self.current_price}
        Stop Loss: ₹{self.stop_loss}
        Highest Price: ₹{self.highest_price}
        Unrealized P&L: ₹{unrealized_pnl:,.2f}
        Points from Stop Loss: {abs(self.current_price - self.stop_loss):.2f}
        {'='*60}
        """
        return status


# Example Usage
def main():
    """
    Main function demonstrating the trailing stop loss system
    """
    
    # Initialize Kite Connect
    api_key = "your_api_key"
    access_token = "your_access_token"
    
    kite = KiteConnect(api_key=api_key)
    kite.set_access_token(access_token)
    
    # Initialize Kite Ticker
    kite_ticker = KiteTicker(api_key, access_token)
    
    # Get Nifty December Future instrument token
    # Example token - get actual from instruments list
    instruments = kite.instruments("NFO")
    nifty_dec = [inst for inst in instruments 
                 if inst['tradingsymbol'] == 'NIFTY24DECFUT'][0]
    instrument_token = nifty_dec['instrument_token']
    
    # Initialize trailing stop loss system
    tsl = TrailingStopLoss(kite, kite_ticker, instrument_token)
    
    # Enter position
    # Example: Buy at 20000 with 15 points stop loss and 15 points trailing
    tsl.enter_position(
        entry_price=20000,
        quantity=75,  # 1 lot of Nifty = 75 quantity
        position_type='BUY',
        initial_stop_distance=15,
        trailing_points=15
    )
    
    # Ticker callbacks
    def on_ticks(ws, ticks):
        """Process incoming ticks"""
        for tick in ticks:
            if tick['instrument_token'] == instrument_token:
                current_price = tick['last_price']
                
                # Update trailing stop loss
                tsl.update_trailing_stop(current_price)
                
                # Log current status every 30 seconds (optional)
                # logger.info(tsl.get_position_status())
    
    def on_connect(ws, response):
        """Subscribe on connection"""
        logger.info("WebSocket connected")
        ws.subscribe([instrument_token])
        ws.set_mode(ws.MODE_QUOTE, [instrument_token])
    
    def on_close(ws, code, reason):
        """Handle connection close"""
        logger.info(f"Connection closed: {reason}")
    
    def on_error(ws, code, reason):
        """Handle errors"""
        logger.error(f"Error: {reason}")
    
    # Assign callbacks
    kite_ticker.on_ticks = on_ticks
    kite_ticker.on_connect = on_connect
    kite_ticker.on_close = on_close
    kite_ticker.on_error = on_error
    
    # Start ticker
    logger.info("Starting trailing stop loss system...")
    kite_ticker.connect()


if __name__ == "__main__":
    main()